In [157]:
from pywr.core import Model
from pywr.notebook import draw_graph
import matplotlib.pyplot as plt
import os
from pywr.domains.groundwater import KeatingAquifer
from pywr.parameters.groundwater import KeatingStreamFlowParameter
from pywr.core import Model, Input, Output, Link, Storage, Node, Catchment
from pywr.recorders import (
    NumpyArrayNodeRecorder,
    NumpyArrayLevelRecorder,
    NumpyArrayStorageRecorder,
)
import pandas
import numpy as np
import pytest

def createAquifer(model,name):
    num_streams = 1
    num_additional_inputs = 0
    stream_flow_levels = [[1.0, 10.0]]  # m
    transmissivity = [100, 2000]  # m2/d
    transmissivity = [t * 0.0001 for t in transmissivity]  # m3 to Ml
    coefficient = 1  # no units
    storativity = [0.000005]  # % este parámetro es clave
    levels = [1.0, 10.0]  # m
    area = 5000 * 5000  # m2
    
    aqfer=KeatingAquifer(
        model,
        name,
        num_streams,
        num_additional_inputs,
        stream_flow_levels,
        transmissivity,
        coefficient,
        levels,
        area=area,
        storativity=storativity)

    aqfer.initial_level=1
    
    return aqfer

def connectAquifer(modelo,acuifero,nodosRecarga,nodosDescarga):
    for nodoR in nodosRecarga:
        model.nodes[nodoR].connect(acuifero)
    
    for nodoD in nodosDescarga:
        acuifero.connect(modelo.nodes[nodoD],from_slot=0)
    
    return modelo,acuifero
    
def connectNodes(modelo,entradas,salidas):
    
    for ind,entrada in enumerate(entradas):
        modelo.nodes[entrada].connect(model.nodes[salidas[ind]])
           
    return modelo

if __name__ == "__main__":
    
    path="nuble.json"
    model = Model.load(path)
    
#     crear objetos acuíferos
    aqferNiquen=createAquifer(model,'AC01_NIQ')
    aqferChan=createAquifer(model,'AC02_CHA')
    aqferNuble=createAquifer(model,'AC03_NUB')
    
#     conectar nodos existentes al acuifero Ñiquen
    recargas=['REMZemitaSub','REMVirguinJFRSub','REMSanPedroSub']
    descargas=['abstraccionAqNiq','ZemitaMix','VirguinJFRMix','deficitSanPedro']
    model,aqferNiquen=connectAquifer(model,aqferNiquen,
recargas,descargas)

#     conectar nodos existentes al acuifero Changaral
    recargas=['REMGYMSub','REMMunicipalSub','REMSACSub','REMCSSub',
'REMRanchilloSub','REMGaonaPeralesSub','REMArrauNiquenSub',
'REMLurinSilvaSub','REMSantaSaraSub','REMPomuyetoAltoSub','REMSantaMartaSub',
'REMMoreiraSub','REMLilahueSub']
    
    descargas=['abstraccionAqCha','GreeneYMairaMix','deficitMunicipal',
'deficitCS','deficitRanchillo','deficitGaonaPerales','deficitArrauNiquen',
'deficitLurinSilva','deficitSantaSara','deficitPomuyetoAlto','deficitSantaMarta',
'deficitMoreira','deficitLilahue']
    
    model,aqferChan=connectAquifer(model,aqferChan,
recargas,descargas)
    
#     conectar nodos existentes al acuifero Ñuble
    recargas=['REMCollicoSub','REMChacayalOSub','REMChacayalPSub',
'REMMuticuraSub','REMBellavistaSub','REMMonteBlancoBSub','REMPomuyetoBajoSub','REMLasCulebrasSub']
    
    descargas=['abstraccionAqNub','deficitCollico','deficitChacayalO',
'deficitChacayalP','deficitMuticura','deficitBellavista','deficitMonteBlancoB','deficitPomuyetoBajo',
'deficitLasCulebras']
    
    model,aqferNiquen=connectAquifer(model,aqferNuble,
recargas,descargas)
        
#     conectar los nodos de déficit que faltan
    entradas=['deficitCollico','deficitChacayalO','deficitChacayalP',
'deficitMunicipal','deficitCS','deficitRanchillo','deficitGaonaPerales',
'deficitSanPedro','deficitArrauNiquen','deficitLurinSilva','deficitMuticura',
'deficitSantaSara','deficitPomuyetoAlto','deficitSantaMarta','deficitMoreira',
'deficitLilahue','deficitMonteBlancoB','deficitPomuyetoBajo','deficitLasCulebras']
    
    salidas=['CollicoMix','ChacayalOMix','ChacayalPMix','MunicipalMix','CaroSaldanaMix',
'RanchilloMix','GaonaPeralesMix','SanPedroMix','ArrauNiquenMix',
'LurinSilvaMix','MuticuraMix','SantaSaraMix','PomuyetoAltoMix','SantaMartaMix',
'MoreiraMix','LilahueMix','MonteBlancoBMix','PomuyetoBajoMix','LasCulebrasMix']

#     conectar los sitios de bombeo de aguas subterráneas
    model=connectNodes(model,entradas,salidas)

#     correr modelo
    model.solver.name="glpk-edge"
    stats = model.run()
    print(stats)
    
    plt.figure()
    draw_graph(model,labels=True,attributes=True,
               width=950,height=950)
    plt.show()


Model executed 1 scenarios in 59.8 seconds, running at 351.8 timesteps per second.


<IPython.core.display.Javascript object>

<Figure size 640x480 with 0 Axes>

In [98]:
# bocatoma 3
print((1000-model.nodes["bocatoma3"].flow)/2)
#validar el balance de masa en canal Virguin
print(model.nodes["FRVirguin"].flow)
print(model.nodes["REMVirguinJFR"].flow)
#validar balance de masa en canal Collico
print(model.nodes["CollicoSup"].flow)
print(model.nodes["CollicoMix"].flow)
print(model.nodes["REMCollico"].flow)
print(model.nodes["deficitCollico"].flow)
print(model.nodes["FRCollico"].flow)
print(model.nodes["bocatoma3"].flow)
print(model.nodes["bocatoma4"].flow)


[14.37571429]
[28.57142857]
[0.]
[14.28571429]
[14.28571429]
[0.]
[0.]
[28.75142857]
[971.24857143]
[942.49714286]


In [2]:
# from pywr.notebook import to_html
# to_html(m,height=800,width=800,labels=True,attributes=True,title='Modelo Cuenca del Río Huasco')

In [10]:
from pywr.notebook import pywr_model_to_d3_json
d3json=pywr_model_to_d3_json(model)
import json
with open("./d3.json", "w") as fp:
    json.dump(d3json,fp) 